In [1]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from fastapi import FastAPI
from langserve import add_routes

In [2]:
load_dotenv()

True

In [3]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1
)

messages = [
    SystemMessage(content="Translate the following from English to Turkish."),
    HumanMessage(content="My best book genre is fantastic fiction.")
]

response = llm.invoke(messages)
response

AIMessage(content='En sevdiğim kitap türü fantastik kurgu.', response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 27, 'total_tokens': 39}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f33667828e', 'finish_reason': 'stop', 'logprobs': None}, id='run-1da54395-0702-4822-bcc8-a7418db9b31b-0', usage_metadata={'input_tokens': 27, 'output_tokens': 12, 'total_tokens': 39})

### **Output Parser Nedir?**

**Output parser**, bir modelden elde edilen ham çıktıların anlamlı ve kullanılabilir bir forma dönüştürülmesini sağlayan bir bileşendir. Genellikle, dil modelleri (örneğin, metin üretimi yapan modeller) belirli bir formatta veri üretir, ancak bu çıktıyı uygulamanızın gereksinimlerine uygun hale getirmek için ek bir işleme ihtiyaç duyabilirsiniz.

### **LangChain'de Output Parser'ın Rolü**

LangChain'de output parser, genellikle şu görevleri üstlenir:

1. **Çıktıyı Formatlama:** Modelin ürettiği metni, belirli bir yapı veya formatta düzenler. Örneğin, modelden alınan serbest biçimli metni JSON veya CSV gibi yapılandırılmış bir formata dönüştürebilir.

2. **Veri Çıkarma:** Modelin yanıtından belirli bilgileri çıkarmak için kullanılır. Örneğin, bir metin içinde tarihleri, isimleri veya diğer önemli verileri ayıklamak.

3. **Hata Yönetimi:** Model çıktısındaki hataları veya anormallikleri tespit edip düzeltebilir. Bu, modelin ürettiği verinin doğruluğunu artırır.

4. **Veri Doğrulama:** Çıktının belirli bir şemaya veya kurallara uygun olup olmadığını kontrol eder.


### **Özet**

Output parser, LangChain'de model çıktılarının anlamlı ve kullanılabilir bir biçimde işlenmesini sağlayan önemli bir bileşendir. Bu araçlar, model sonuçlarının uygulama gereksinimlerine uygun hale getirilmesine yardımcı olur ve veri işleme sürecini daha verimli hale getirir.

In [6]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1
)

messages = [
    SystemMessage(content="Translate the following from English to Turkish."),
    HumanMessage(content="My best book genre is fantastic fiction.")
]

parser = StrOutputParser()
response = llm.invoke(messages)

parser.invoke(response)

'En sevdiğim kitap türü fantastik kurgu.'

In [7]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1
)

messages = [
    SystemMessage(content="Translate the following from English to Turkish."),
    HumanMessage(content="My best book genre is fantastic fiction.")
]

parser = StrOutputParser()
chain = llm | parser

chain.invoke(messages)

'En sevdiğim kitap türü fantastik kurgu.'

LangChain'de **`ChatPromptTemplate`** sınıfı, bir dil modeline (LLM) gönderilecek isteklerin biçimlendirilmesini ve yapılandırılmasını sağlar. `ChatPromptTemplate`, dil modelinin belirli bir formatta yanıt vermesini ve verilen şablonlara göre çalışmasını kolaylaştırır.

### **`ChatPromptTemplate` Nedir?**

`ChatPromptTemplate`, dil modeline bir veya birden fazla mesaj türünde bilgi sağlamanın bir yolunu sunar. Bu mesajlar genellikle iki bölümden oluşur:

1. **Sistem Mesajları (System Messages):** Modelin genel talimatlarını veya görev tanımlarını içerir. Bu mesajlar, modelin görevini nasıl yerine getirmesi gerektiğini belirler.

2. **Kullanıcı Mesajları (User Messages):** Modelin yanıtlaması gereken spesifik bilgi veya verileri içerir. Kullanıcıdan alınan bu veriler, modelin belirli bir çıktıyı üretmesini sağlar.

### **`ChatPromptTemplate` Kullanımı**

**1. Şablon Oluşturma:**

`ChatPromptTemplate`, mesajları şablon şeklinde yapılandırmanıza olanak tanır. Şablonlar genellikle metin içinde yer tutucular kullanır. Örneğin, `{language}` veya `{text}` gibi yer tutucular, dinamik olarak değiştirilebilecek yerleri temsil eder.

**2. Şablonun Kullanılması:**

Şablonlar, dil modeline belirli bir formatta bilgi sağlamak için kullanılır. Model bu şablona göre yanıt üretir. Şablon, modelin istenen formatta ve içeriğe uygun şekilde yanıt vermesine yardımcı olur.

**Örnekle Açıklama:**

Aşağıda verilen kod örneğinde olduğu gibi, şablonlar şu şekilde çalışır:

1. **Sistem Mesajı:** Modelin görevini tanımlar. Örneğin, "Translate following into {language}" mesajı, modelden verilen metni belirli bir dile çevirmesi istenir.

2. **Kullanıcı Mesajı:** Modelin işlem yapacağı asıl veriyi içerir. Örneğin, `{text}` yer tutucusu, çevrilecek metni temsil eder ve bu metin şablona yerleştirilir.

**Temel Adımlar:**

1. **Şablonun Tanımlanması:** Şablon, modelin yanıt vermesi için gereken genel talimatları ve spesifik verileri içerir. Şablondaki yer tutucular, dinamik verilerle doldurulacaktır.

2. **Şablonun Uygulanması:** Şablon oluşturulduktan sonra, gerçek verilerle doldurulur ve dil modeline gönderilir. Model, şablona göre formatlanmış veriyi alır ve yanıt üretir.

### **Özet**

`ChatPromptTemplate`, dil modellerine verilecek isteklerin ve yanıtların yapılandırılmasını sağlayan bir şablondur. Bu şablonlar, modelin belirli bir formatta ve içeriğe uygun yanıtlar üretmesini sağlar. Şablonlar, genellikle sistem ve kullanıcı mesajları olarak ikiye ayrılır ve dinamik yer tutucular kullanarak çeşitli veri türlerini işleyebilir.

In [8]:
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1
)

system_prompt = "Translate following into {language}"
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("user", "{text}")
    ]
)


parser = StrOutputParser()

chain = prompt_template | llm | parser # prompt_template i al modele ver, çıktıyı al parser a ver

chain.invoke({"language": "Italian", "text": "Hello World!"})

'Ciao Mondo!'

# **LangServe Kullanımı**

In [3]:
import nest_asyncio

In [4]:
from fastapi import FastAPI
from langserve import add_routes
import uvicorn



llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1
)

system_prompt = "Translate following into {language}"
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("user", "{text}")
    ]
)


parser = StrOutputParser()

chain = prompt_template | llm | parser # prompt_template i al modele ver, çıktıyı al parser a ver

app = FastAPI(
    title="Translator App",
    version="1.0.0",
    description="Translation Chat Bot"
)

add_routes(
    app,
    chain,
    path="/chain"
)

nest_asyncio.apply()

uvicorn.run(app, host="localhost", port=8000)

INFO:     Started server process [37788]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://localhost:8000 (Press CTRL+C to quit)



 __          ___      .__   __.   _______      _______. _______ .______     ____    ____  _______
|  |        /   \     |  \ |  |  /  _____|    /       ||   ____||   _  \    \   \  /   / |   ____|
|  |       /  ^  \    |   \|  | |  |  __     |   (----`|  |__   |  |_)  |    \   \/   /  |  |__
|  |      /  /_\  \   |  . `  | |  | |_ |     \   \    |   __|  |      /      \      /   |   __|
|  `----./  _____  \  |  |\   | |  |__| | .----)   |   |  |____ |  |\  \----.  \    /    |  |____
|_______/__/     \__\ |__| \__|  \______| |_______/    |_______|| _| `._____|   \__/     |_______|

LANGSERVE: Playground for chain "/chain/" is live at:
LANGSERVE:  │
LANGSERVE:  └──> /chain/playground/
LANGSERVE:
LANGSERVE: See all available routes at /docs/

INFO:     ::1:56973 - "GET /chain HTTP/1.1" 404 Not Found
INFO:     ::1:56973 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     ::1:57004 - "GET /chain/playground HTTP/1.1" 307 Temporary Redirect
INFO:     ::1:57004 - "GET /chain/playground/ HTT